In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from typing import Optional, List,Tuple,Dict

C:\Users\Łukasz\AppData\Roaming\Python\Python39\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


# Phone Ranking Problem

Choosing a new phone is often quite a daunting task, with many different models from different companies and varying prices.

In [2]:
data = pd.read_csv("./data/dataset.csv")
prefs = pd.read_csv("./data/preferences.csv")

In [3]:
data

,name,price-cost,ram-gain,screen-gain,disk-gain,oled-gain,os-gain,foldable-gain
0,Huawei P30,1699,6.000,6.10,128,1,3,0
1,Huawei P20 Lite,899,4.000,5.80,64,0,3,0
2,Iphone 15,4699,6.000,6.70,128,1,0,0
3,Iphone 12,2499,4.000,6.10,64,1,0,0
4,Motorola Razr Ultra,3999,8.000,6.90,256,1,3,1
5,Motorola Razr 40,2999,8.000,6.90,256,1,3,1
6,Samsung Galaxy Z Flip 5,5599,8.000,6.70,256,1,3,1
7,Motorola Razr 8,2657,8.000,6.67,256,1,3,1
8,Nokia 2660 Flip,299,4.000,2.80,128,1,1,1
9,Huawei P40,1949,8.000,6.10,128,1,2,0


In [4]:
crit_types = [True if crit.split("-")[1]=="gain" else False for crit in data.drop(columns="name").columns ]
crit_types

[False, True, True, True, True, True, True]

In [5]:
data.columns = list(map(lambda s: s.split("-")[0],data.columns))
data

,name,price,ram,screen,disk,oled,os,foldable
0,Huawei P30,1699,6.000,6.10,128,1,3,0
1,Huawei P20 Lite,899,4.000,5.80,64,0,3,0
2,Iphone 15,4699,6.000,6.70,128,1,0,0
3,Iphone 12,2499,4.000,6.10,64,1,0,0
4,Motorola Razr Ultra,3999,8.000,6.90,256,1,3,1
5,Motorola Razr 40,2999,8.000,6.90,256,1,3,1
6,Samsung Galaxy Z Flip 5,5599,8.000,6.70,256,1,3,1
7,Motorola Razr 8,2657,8.000,6.67,256,1,3,1
8,Nokia 2660 Flip,299,4.000,2.80,128,1,1,1
9,Huawei P40,1949,8.000,6.10,128,1,2,0



1. What is the domain of the problem about?
	
	The problem involves constinst of various phones and some of their parameters. 
	Price is given in złoty's, Ram in GB, Screen in inches, disk in GB, Oled and Foldable are binary variables and its beneficial for the phone to have them.
	OS ranks the operating systems with 0 being iOS, 1 being other, 2 being HarmonyOS and 3 being Android.

2. What is the source of the data?

	The data is taken from the internet from various online shops that offer these phones.

In [6]:
def type_1(d):
	return 1 if d > 0 else 0

def type_5(d,q:float,p:float):
	if d > p:
		return 1
	elif d <= q:
		return 0
	else: 
		return (d-q)/(p-q)

In [80]:
def rank_string(ranking,names,third_column=None,third_name="id"):
	_str = f'{"rank":<5} | {"name":30} | {third_name:<5}\n'
	_str += len(_str)*"="
	_str += "\n"
	# for i,n in sorted(zip(ranking,names),key=lambda a:a[0]):
	for i,_id in enumerate(ranking):
		n = names[_id]
		_str += f"{i:<5} | {n:<30} | {_id if third_column is None else third_column[_id]:<5}\n"
	return _str

def show_rank(ranking,names):
	print(rank_string(ranking,names))

## SRF

In [8]:
def srf(rank:List[Optional[Tuple[str]]])->Dict[str,float]:
    """
        blank cards are none
    """

    n = len(rank)
    z = n-1
    blanks_below = rank.count(None)
    t = n-blanks_below
    r = []

    for i,elem in enumerate(rank):
        if elem is None:
            continue
        r.append(n-i)
    
    weights = np.array([1+((z-1)*((r_t-1)/(r[0]-1))) for r_t in r])

    criterions = filter(lambda a: not a is None,rank)
    mapping = {}
    for i,crits in enumerate(criterions):
        for crit in crits:
            mapping[crit] = weights[i]
    
    _sum = sum(mapping.values())
    mapping = dict([(k,v/_sum) for k,v in mapping.items()])

    return mapping

In [9]:
ranking = [
    ("price",),
    None,
    None,
    ("os",),
    ("foldable",),
    None,
    ("oled","screen"),
    ("ram","disk")]

In [10]:
crit_weights = srf(ranking)
print(crit_weights)
crit_weights = list(map(lambda c: crit_weights[c],data.drop(columns="name").columns))

{'price': 0.33793103448275863, 'os': 0.21379310344827587, 'foldable': 0.17241379310344826, 'oled': 0.0896551724137931, 'screen': 0.0896551724137931, 'ram': 0.04827586206896552, 'disk': 0.04827586206896552}


## PROMETHEE

In [72]:
class Promethee:
	def __init__(self,data:pd.DataFrame,criterion_type:Tuple[bool],criterion_weights:Tuple[float]=None,discrimination_thresholds:np.array=None):
		self.data = data.drop(columns="name")
		self.names = data["name"]
		self.ranking = [i for i in range(len(self.names))]
		self.is_crit_gain = criterion_type
		self.criterion_weighs = criterion_weights
		self.criterion_funcs = self._init_crit_funcs(discrimination_thresholds)

	def _init_crit_funcs(self,disc_thresh):
		output=[]
		for i,(p,q) in enumerate(disc_thresh):
			output.append((lambda qi,pi: lambda a,b: type_5(a-b,qi,pi))(q,p) if self.is_crit_gain[i] else (lambda qi,pi: lambda a,b: type_5(b-a,qi,pi))(q,p))
		return output
		

	def _cp_index(self,data:pd.DataFrame,criterion_weights:List[float]=None,criterion_func:List[callable]=None)->np.array:

		n = len(data)
		matrix = np.zeros((n,n))

		if criterion_weights==None:
			criterion_weights = [1 for _ in data.columns]

		if criterion_func == None:
			criterion_func = [type_1 for _ in criterion_weights]

		for c,criterion in enumerate(data.columns):
			for i,a in enumerate(data[criterion]):
				for j,b in enumerate(data[criterion][i+1:],i+1):
					pi = criterion_func[c]
					w = criterion_weights[c]
					matrix[i][j] += w * pi(a,b)
					matrix[j][i] += w * pi(b,a)
		
		return matrix/sum(criterion_weights)
	
	def _flow(self,cpi:np.array):
		positive_flow = np.sum(cpi,axis=1)
		negative_flow = np.sum(cpi,axis=0)

		return positive_flow,negative_flow
	
	def _flip_rank(self,rank):
		flipped = [None for id in rank]
		for r,id in enumerate(rank):
			flipped[id] = r
		return flipped

	def __str__(self):
		return rank_string(self.ranking,self.names)

In [73]:
class Promethee1(Promethee):
	def rank(self):
		cpi = self._cp_index(self.data,self.criterion_weighs,self.criterion_funcs)
		self.pos_flow,self.neg_flow = self._flow(cpi)

		self.pos_rank = np.argsort(-self.pos_flow)
		self.neg_rank = np.argsort(self.neg_flow)

		# ranking = [[] for _ in range(len(self.data))]
		# for i,p in enumerate(self.pos_flow):
		# 	for j,n in enumerate(self.neg_flow[i+1:],i+1):
		# 		if not self._equal_check(i,j):
		# 			continue
		# 		ranking[i].append(j)
		# print(ranking)

		return self.ranking

	def __str__(self):
		return f"Positive flow ranking:\n{self.positive_ranking()}\nNegative flow ranking:\n{self.negative_ranking()}"
	
	def positive_ranking(self):
		return rank_string(self.pos_rank,self.names)
	
	def negative_ranking(self):
		return rank_string(self.neg_rank,self.names)

	def _equal_check(self,i,j):
		return ((np.allclose(self.pos_flow[i],self.pos_flow[j]) and np.allclose(self.neg_flow[i],self.neg_flow[j])) or 
		(self.pos_flow[i] > self.pos_flow[j] and self.neg_flow[i] > self.neg_flow[j]) or 
		(self.pos_flow[i] < self.pos_flow[j] and self.neg_flow[i] < self.neg_flow[j]))

In [74]:
class Promethee2(Promethee):
	def rank(self):
		self.cpi = self._cp_index(self.data,self.criterion_weighs,self.criterion_funcs)
		pos_flow,neg_flow = self._flow(self.cpi)

		self.f = pos_flow-neg_flow

		self.ranking = np.argsort(-self.f)

		return self.ranking

In [14]:
crit_discrimination = [
	[500,100],
	[4,2],
	[0.5,0.1],
	[64,16],
	[0,0],
	[2,0],
	[0,0]
]

In [75]:
prom1ranker = Promethee1(data,crit_types,crit_weights,crit_discrimination)
prom1ranker.rank()
print(prom1ranker.positive_ranking())

rank  | name                           | id   
0     | Nokia 2660 Flip                | 8    
1     | Samsung Galaxy M34             | 12   
2     | Xiaomi Poco X6 Pro             | 13   
3     | Motorola Razr 8                | 7    
4     | Motorola Razr 40               | 5    
5     | Motorola Razr Ultra            | 4    
6     | Huawei P20 Lite                | 1    
7     | Lg V60                         | 14   
8     | Nokia 3310                     | 11   
9     | Samsung Galaxy Z Flip 5        | 6    
10    | Huawei P30                     | 0    
11    | Samsung Galaxy A55             | 10   
12    | Huawei P40                     | 9    
13    | Iphone 12                      | 3    
14    | Iphone 15                      | 2    



In [16]:
print(prom1ranker.negative_ranking())

rank  | name                           | id   
0     | Samsung Galaxy M34             | 12   
1     | Xiaomi Poco X6 Pro             | 13   
2     | Lg V60                         | 14   
3     | Motorola Razr 8                | 7    
4     | Huawei P30                     | 0    
5     | Samsung Galaxy A55             | 10   
6     | Motorola Razr 40               | 5    
7     | Nokia 2660 Flip                | 8    
8     | Motorola Razr Ultra            | 4    
9     | Huawei P20 Lite                | 1    
10    | Samsung Galaxy Z Flip 5        | 6    
11    | Huawei P40                     | 9    
12    | Nokia 3310                     | 11   
13    | Iphone 12                      | 3    
14    | Iphone 15                      | 2    



In [17]:
prom2ranker = Promethee2(data,crit_types,crit_weights,crit_discrimination)
prom2ranker.rank()
print(str(prom2ranker))


rank  | name                           | id   
0     | Samsung Galaxy M34             | 12   
1     | Xiaomi Poco X6 Pro             | 13   
2     | Nokia 2660 Flip                | 8    
3     | Lg V60                         | 14   
4     | Motorola Razr 8                | 7    
5     | Motorola Razr 40               | 5    
6     | Motorola Razr Ultra            | 4    
7     | Huawei P30                     | 0    
8     | Samsung Galaxy A55             | 10   
9     | Huawei P20 Lite                | 1    
10    | Samsung Galaxy Z Flip 5        | 6    
11    | Huawei P40                     | 9    
12    | Nokia 3310                     | 11   
13    | Iphone 12                      | 3    
14    | Iphone 15                      | 2    



## ELECTRE-TRI-B

In [82]:
class ElectreTriB:
	def __init__(self,data:pd.DataFrame,criterion_type:list[bool],criterion_weights:list[float],class_profiles:np.array, pref_indif_veto:np.array):
		"""
			class_profiles should be (criterion x profiles)

			pref_indif_veto should be(criterion x profiles x 3)
		"""
		self.data = data.drop(columns="name")
		self.names = data["name"]
		self.is_crit_gain = criterion_type
		self.class_profiles = class_profiles
		self.pref_indif_veto = pref_indif_veto
		self.weights = criterion_weights

		self.b = self.class_profiles.shape[1]
		self.c = self.b-1
		self.g = len(self.data.columns)
		self.a = len(self.data)

	def _marginals(self,thresholds,isConc=True,isInverse=False):
		matrix = np.zeros((self.g,self.b,self.a)) if not isInverse else np.zeros((self.g,self.a,self.b))

		for g_i,g in enumerate(self.data.columns):
			for a_i in range(self.a):
				for b_j in range(self.b):

					# for gain criterion
					d = self.class_profiles[g_i][b_j]-self.data[g][a_i]
					
					# invert for cost
					if not self.is_crit_gain[g_i]:
						d *= -1

					# invert for profile first, alternative second
					if isInverse:
						d *= -1

					qv,p = thresholds[g_i][b_j],self.pref_indif_veto[g_i][b_j][0]
					out = 0
					if isConc:
						out = self._conc(d,qv,p)
					else:
						out = self._disc(d,qv,p)

					if not isInverse:
						matrix[g_i][b_j][a_i] = out
					else:
						matrix[g_i][a_i][b_j] = out

		return matrix

	def _marginal_concordance(self):
		self.mconc = self._marginals(self.pref_indif_veto[:,:,1])
	def _inv_m_conc(self):
		self.invmconc = self._marginals(self.pref_indif_veto[:,:,1],isInverse=True)

	def _conc(self,d,q,p):
		if d >= p:
			return 0
		elif d <= q:
			return 1 
		else:
			return (p-d)/(p-q)
	
	def _disc(self,d,v,p):
		if d <= p:
			return 0
		elif d >= v:
			return 1
		else:
			return (d-p)/(v-p)

	def _comprehensive_concordance(self,mconc):
		return np.sum((np.array(self.weights)*mconc.T).T,axis=0)/np.sum(self.weights)

	def _marginal_discordance(self):
		self.mdisc = self._marginals(self.pref_indif_veto[:,:,2],isConc=False)
	def _inv_m_disc(self):
		self.invmdisc = self._marginals(self.pref_indif_veto[:,:,2],isConc=False,isInverse=True)

	def _outranking_credibility(self):
		self.out_cred = np.zeros((self.a,self.b))
		self.inv_out_cred = np.zeros((self.b,self.a))

		for a_i in range(self.a):
			for b_j in range(self.b):
				_C = self.cconc[b_j][a_i]
				_invC = self.invcconc[a_i][b_j]
				f = [f_j for f_j in self.mdisc[:,b_j,a_i] if f_j > _C]
				invf = [f_j for f_j in self.invmdisc[:,a_i,b_j] if f_j > _invC]
				#f.append(0)
				prod = np.prod([(1-f_j)/(1-_C) for f_j in f])
				invprod = np.prod([(1-f_j)/(1-_invC) for f_j in invf])

				self.out_cred[a_i][b_j] = _C * prod
				self.inv_out_cred[b_j][a_i] = _invC * invprod

	def class_assignment(self,cred_threshold,isPessimistic=True):
		self.run()

		classes = np.zeros(self.a,dtype=np.int16) if isPessimistic else np.full(self.a,self.c-1)

		for a_i in range(self.a):
			for b_j in range(self.b):
				if isPessimistic: b_j = self.b - b_j - 1

				if isPessimistic and self._pessimistic_ev(a_i,b_j,cred_threshold):
					classes[a_i]=(max(min(b_j+1,self.c-1),0))
					break

				if (not isPessimistic) and self._optimistic_ev(a_i,b_j,cred_threshold):
					classes[a_i]=(max(min(b_j,self.c-1),0))	
					break
		
		return classes

	def _pessimistic_ev(self,a,b,thresh):
		return self.out_cred[a][b] >= thresh# and self.inv_out_cred[b][a] < thresh
	
	def _optimistic_ev(self,a,b,thresh):
		return self.out_cred[a][b] < thresh and self.inv_out_cred[b][a] >= thresh

	def show_assignments(self,thresh):
		print("Optimistic class assignment:")
		optimistic = self.class_assignment(thresh,isPessimistic=False)
		default_rank = [i for i in range(self.a)]
		print(rank_string(default_rank,self.names,optimistic,"class"))
		print("Pessimistic class assignment:")
		pessimistic=self.class_assignment(thresh,isPessimistic=True)
		print(rank_string(default_rank,self.names,pessimistic,"class"))

	def run(self):
		self._marginal_concordance()
		self._inv_m_conc()
		self._marginal_discordance()
		self._inv_m_disc()
		self.cconc = self._comprehensive_concordance(self.mconc)
		self.invcconc = self._comprehensive_concordance(self.invmconc)
		self._outranking_credibility()

In [83]:
class_profiles = np.array([
	#[500,0.2,0.5,2,0,0,0],
	[0,0,0,0,0,0,0],
	[1000,0.5,2,10,0,1,0],
	[2500,3,4.5,100,1,2,1],
	[3700,7,5.9,350,1,3,1],
	[5000,16,7,640,1,3,1]
]).T

preferences = [
	[500,1000,1000,1000,1000],
	[2,2,2,4,4],
	[3,0.5,0.5,0.2,0.2],
	[3,4,16,32,32],
	[1,1,0,0,0],
	[3,2,1,0,0],
	[1,1,1,0,0]
]

indifferences = [
	[100,100,150,200,500],
	[1,1,1,1,2],
	[1,0.3,0.25,0.1,0.1],
	[1,2,8,16,16],
	[0,0,0,0,0],
	[1,1,0,0,0],
	[0,0,0,0,0]
]

vetoes = [
	[6000,1000,2000,2000,2000],
	[4,8,8,16,16],
	[6,1,1,0.5,0.5],
	[8,16,32,64,64],
	[2,2,2,2,2],
	[2,2,1,1,1],
	[2,2,2,2,2]
]

pref_indif_vetoes = np.stack([preferences,indifferences,vetoes],axis=2)

In [84]:
electre = ElectreTriB(data,crit_types,crit_weights,class_profiles,pref_indif_vetoes)

electre.show_assignments(0.7)

Optimistic class assignment:
rank  | name                           | class
0     | Huawei P30                     | 3    
1     | Huawei P20 Lite                | 3    
2     | Iphone 15                      | 3    
3     | Iphone 12                      | 3    
4     | Motorola Razr Ultra            | 3    
5     | Motorola Razr 40               | 3    
6     | Samsung Galaxy Z Flip 5        | 3    
7     | Motorola Razr 8                | 3    
8     | Nokia 2660 Flip                | 3    
9     | Huawei P40                     | 3    
10    | Samsung Galaxy A55             | 3    
11    | Nokia 3310                     | 3    
12    | Samsung Galaxy M34             | 3    
13    | Xiaomi Poco X6 Pro             | 3    
14    | Lg V60                         | 3    

Pessimistic class assignment:
rank  | name                           | class
0     | Huawei P30                     | 3    
1     | Huawei P20 Lite                | 2    
2     | Iphone 15                      | 0    
